# BigQuery

**NOTE**: None of the datasets are found in this repository. They're large and I didn't feel like taking them from Kaggle's website just to be able to run the code. 

In [ ]:
import bq_helper
# create a helper object for our bigquery dataset
chicago_crime = bq_helper.BigQueryHelper(active_project= "bigquery-public-data", 
                                       dataset_name = "chicago_crime")

## Get a list of the tables in the BigQuery Dataset

In [ ]:
chicago_crime.list_tables()

## Look at one tables schema and check out the head

The columns represent (from left to right):
- The name of the column
- The datatype of the column
- The mode of the column (NULLABLE means that a column allows NULL values, and is the default)
- A description of the data in that column

In [ ]:
chicago_crime.table_schema("crime")

In [ ]:
chicago_crime.head("crime", selected_columns=["date", "description", "arrest"], num_rows=20)

## Check the size of an SQL query before executing it!

This can be important because, in large datasets, scanning a lot of data can be very costly. For example, Kaggle let's users scan 5 TB of data every 30 days.

__NOTE__: I have no idea what estimate_query_size returns and I can't find documentation for bq_helper. From context clues of the lesson I think the value returned is in units of GBs. Also, this returns the estimated size of the data returned from the query. It does not tell you **anything** about the amount of the data that must be scanned.

### Running the Query
- *BigQueryHelper.query_to_pandas(query)*: This method takes a query and returns a Pandas dataframe.
- *BigQueryHelper.query_to_pandas_safe(query, max_gb_scanned=1)*: This method takes a query and returns a Pandas dataframe only if the size of the query is less than the upperSizeLimit (1 gigabyte by default).

In [ ]:
# This query looks at the crime table of the Chicago crime data set
# It scans for all descriptions where arrest is False
query = """SELECT description
            FROM `bigquery-public-data.chicago_crime.crime`
            WHERE arrest = False
        """
chicago_crime.estimate_query_size(query)

In [ ]:
arrested_descriptions = chicago_crime.query_to_pandas_safe(query, max_gb_scanned=0.12)

In [ ]:
arrested_descriptions.head()

# SQL Queries: Select, From, Where

This will following along with the SQL exercise provided in the lessons using the openaq dataset.

In [ ]:
import bq_helper
# create a helper object for this dataset
open_aq = bq_helper.BigQueryHelper(active_project="bigquery-public-data",
                                   dataset_name="openaq")

# print all the tables in this dataset (there's only one!)
open_aq.list_tables()

In [ ]:
open_aq.table_schema("global_air_quality")

In [ ]:
open_aq.head("global_air_quality", selected_columns="unit", num_rows=10)

## Question 1: Which counties use a unit other than ppm to measure any type of pullution?

In [ ]:
query = """ SELECT country
            FROM `bigquery-public-data.openaq.global_air_quality`
            WHERE unit != "ppm"
        """

non_ppm_countries = open_aq.query_to_pandas_safe(query)

The countries with the most measurements not using ppm as a unit.


In [ ]:
non_ppm_countries.country.value_counts().head()

## Question 2: Which pollutants have a value of exactly 0?

In [ ]:
query = """SELECT pollutant
            FROM `bigquery-public-data.openaq.global_air_quality`
            WHERE value = 0
        """

zero_value_pollutants = open_aq.query_to_pandas_safe(query)

All pollutants with 0 value counts!

In [ ]:
zero_value_pollutants.pollutant.value_counts()

# SQL Queries: Group By, Having, and Count

- **COUNT**: Returns the number of entries given the name of a column (i.e. COUNT(name) will return the number of entries for the column 'name')
    - *Note*: Count is an example of an **aggregate function** which a type of SQL function that takes in many values and returns one (i.e. SUM() or AVERAGE())
- **GROUP BY**: GROUP BY takes the name of one or more column and tells SQL that we want to treat rows that have the same value in that column as a single group when we apply aggregate functions like COUNT().
- **GROUP BY ... HAVING**: This is the same as GROUP BY, but it enforces a criteria on the groups and only returns groups that meet the criteria.

These exercises will use the hacker news data set.



In [ ]:
import bq_helper

hacker_news = bq_helper.BigQueryHelper(active_project="bigquery-public-data", dataset_name="hacker_news")

hacker_news.list_tables()

## Question 1: How many stories are there of each type in the full table?

In [ ]:
hacker_news.table_schema("full")

In [ ]:
query = """SELECT type, COUNT(id)
            FROM `bigquery-public-data.hacker_news.full`
            GROUP BY type
        """

unique_stories = hacker_news.query_to_pandas_safe(query)

The number of stories for each type

In [ ]:
unique_stories.head()

## Question 2: How many comments have been deleted? 

In [ ]:
hacker_news.table_schema("comments")

In [ ]:
query = """SELECT deleted, COUNT(ID)
            FROM `bigquery-public-data.hacker_news.comments`
            GROUP BY deleted
            HAVING deleted = True
        """

deleted_comments = hacker_news.query_to_pandas_safe(query)

The number of deleted comments.

In [ ]:
deleted_comments.head()

# SQL Queries: Order By and Extract

Order by is typically the last clause one will add into their SQL query because you'll use it to sort the results returned by the rest of the query.

- **Numerical Data**: By default, the column will return the data sorted in lowest to highest. 
- **String Data**: By default, the column will be sorted alphabetically from a-z.

If the **DESC** keyword is specified (i.e. ORDER BY *column_name* DESC) then it will reverse the sort order (i.e. high-low, z-a).

### Dates

Dates can be stored in BigQuery using *DATA* or *DATETIME* format.
- *Date* Format: Year first, month, then day
> `YYYY-[M]M-[D]D`
> - YYYY: Four-digit year
> - [M]M: One or two digit month
> - [D]D: One or two digit day
- *DATETIME/TIMESTAMP* Format: Same as *Date* Format, but the time information is appended
> `YYYY-[M]M-[D]D[( |T)[H]H:[M]M:[S]S[.DDDDDD]][time zone]`
> - YYYY: Four-digit year
> - [M]M: One or two digit month
> - [D]D: One or two digit day
> - ( |T): A space or a T separator
> - [H]H: One or two digit hour (valid values from 00 to 23)
> - [M]M: One or two digit minutes (valid values from 00 to 59)
> - [S]S: One or two digit seconds (valid values from 00 to 59)
> - [.DDDDDD]: Up to six fractional digits (i.e. up to microsecond precision)
> - (TIMESTAMP only) [time zone]: String representing the time zone

Often times, one will only want to look at certain parts of the date (i.e. the year or the day). This can be accomplished using the EXTRACT function as follows:

```
SELECT EXTRACT(DAY FROM column_with_timestamp)
FROM `bigquery....
```
This query will return one column with the day from each date in the column.

```
SELECT EXTRACT(WEEK FROM column_with_timestamp)
FROM `bigquery...`
```
This query will return one column with the week (1-53) from each date in the column.

SQL is really and we can perform advance temporal extractions from dates, for more [see all the functions](https://cloud.google.com/bigquery/docs/reference/legacy-sql) one can use with date in BigQuery under "Data and time functions".



In [ ]:
# import package with helper functions 
import bq_helper

# create a helper object for this dataset
accidents = bq_helper.BigQueryHelper(active_project="bigquery-public-data",
                                   dataset_name="nhtsa_traffic_fatalities")

accidents.list_tables()

In [ ]:
accidents.table_schema("accident_2015")

In [ ]:
accidents.head("accident_2015", selected_columns="number_of_drunk_drivers", num_rows=20)

## Question 1: Which hours of the day do the most accidents occur during?

In [ ]:
# We will get the number of accidents by counting the consecutive number column
# We will get the hour of the day with the EXTRACT function
# We want to group by hour of the day to aggregate the number of accidents at each hour
# and then order them in high-low order
query = """SELECT COUNT(consecutive_number), EXTRACT(HOUR FROM timestamp_of_crash)
            FROM `bigquery-public-data.nhtsa_traffic_fatalities.accident_2015`
            GROUP BY EXTRACT(HOUR FROM timestamp_of_crash)
            ORDER BY COUNT(consecutive_number) DESC
        """

most_accidents_hourly = accidents.query_to_pandas_safe(query)


In [ ]:
most_accidents_hourly
import matplotlib.pyplot as plt
plt.scatter(most_accidents_hourly.f1_, most_accidents_hourly.f0_)
plt.title("Aggregate Hourly Accidents in the US in 2015")
plt.xlabel("Hour of the Day 0-23")
plt.ylabel("Number of Accidents")
plt.show()

## Question 2: Which state has the most drunk drivers?
*Note: This is not the actual question in the lesson, it was original 'Which state has the most hit and run accidents?', but upon investigation the 'hit_and_run' column seems to have been removed from the dataset.*

In [ ]:
# We will SELECT the number of drunk drivers by counting the number of drunk drivers column
# We will SELECT the state name
# We want to GROUP BY the state name to aggregate the number of drunk drivers per state
# and then ORDER BY high-low order
query = """SELECT COUNT(number_of_drunk_drivers), state_name
            FROM `bigquery-public-data.nhtsa_traffic_fatalities.accident_2015`
            GROUP BY state_name
            ORDER BY COUNT(number_of_drunk_drivers) DESC
        """

drunk_drivers_per_state = accidents.query_to_pandas_safe(query)

In [ ]:
import matplotlib.pyplot as plt

state_names = drunk_drivers_per_state.state_name
state_numbs = [numb for numb, name in enumerate(state_names)]

plt.scatter(state_numbs, drunk_drivers_per_state.f0_)
plt.title("Drunk Drivers per State in 2015")
plt.xlabel("State")
plt.xticks(state_numbs, state_names, rotation='vertical', fontsize=9)
plt.ylabel("Number of Drunk Drivers")
plt.show()

## SQL Queries: AS and WITH

AS pretty much allows you to alias columns to new names. See examples below
```
    SELECT EXTRACT(DAY FROM column_with_timestamp), data_point_3
    FROM `bigquery-public-data.imaginary_dataset.imaginary_table`
```
```
    SELECT EXTRACT(DAY FROM column_with_timestamp) AS day,
            data_point_3 AS data
    FROM `bigquery-public-data.imaginary_dataset.imaginary_table`
```
    
Not only does this allow the user to continue using the alias within the query, but it also with rename the resulting columns in the datastructure that is returned from the scan (i.e. the DateFrame).

### Common Table Expressions (CTEs)

One can combine the powerful AS clause with the WITH clause to create WITH...AS clauses. These can then be used to create a CTE.
> **Common table expression**: A temporary table that you return within your query. You can then write queries against the new table you've created. 
> CTE's only exist inside the query where you create them, though, so you can't reference them in later queries.

The following exercises will use the bitcoin dataset.


In [ ]:
# import package with helper functions 
import bq_helper

# create a helper object for this dataset
bitcoin_blockchain = bq_helper.BigQueryHelper(active_project="bigquery-public-data",
                                              dataset_name="bitcoin_blockchain")

bitcoin_blockchain.list_tables()

In [ ]:
bitcoin_blockchain.table_schema("transactions")

## Question 1: How many Bitcoin transactions were made each day in 2017?

In [ ]:
# First, make a CTE with the timestamp in DATETIME format, and the transaction_id
# Then, COUNT the transaction_ids, and create a column for days and years
# Group the data by year such that year must be 2017
query = """WITH time AS
            (
                SELECT TIMESTAMP_MILLIS(timestamp) AS trans_time, transaction_id
                FROM `bigquery-public-data.bitcoin_blockchain.transactions`
            )
            SELECT COUNT(transaction_id) AS transactions, EXTRACT(DAYOFYEAR FROM trans_time) AS abs_day
            FROM time
            WHERE EXTRACT(YEAR FROM trans_time) = 2017
            GROUP BY abs_day
            ORDER BY abs_day            
        """

daily_transactions_2017 = bitcoin_blockchain.query_to_pandas_safe(query, max_gb_scanned=24)

In [ ]:
import matplotlib.pyplot as plt

plt.plot(daily_transactions_2017.abs_day, daily_transactions_2017.transactions)
plt.title("Daily BitCoin Transactions in 2017")
plt.ylabel("Number of Transactions")
plt.xlabel("Day in the Year")
plt.show()

## Question 2: How many transactions are associated with each merkle root?

In [ ]:
query = """
            SELECT COUNT(transaction_id) AS transactions, merkle_root AS root
            FROM `bigquery-public-data.bitcoin_blockchain.transactions`
            GROUP BY root            
        """

transactions_p_mrklRoot = bitcoin_blockchain.query_to_pandas_safe(query, max_gb_scanned=42)

In [ ]:
transactions_p_mrklRoot

## SQL Queries: JOIN

The JOIN clauses is used to combine multiple tables in some way. JOIN represents a large group of clauses depending on how the user would like to JOIN things. For example, the INNER JOIN clause combines only if the element of the column is found in both tables.

## Question 1: How many commits (recorded in the "sample_commits" table) have been made in repos written in the Python programming language? (I'm looking for the number of commits per repo for all the repos written in Python.



In [ ]:
# import package with helper functions 
import bq_helper

# create a helper object for this dataset
github = bq_helper.BigQueryHelper(active_project="bigquery-public-data",
                                              dataset_name="github_repos")

github.list_tables()

In [ ]:
# Table to find commits
github.table_schema("sample_commits")

In [ ]:
# Table to get repos (repo_name) and file names (path)
github.table_schema("sample_files")

In [ ]:
github.head("sample_files", selected_columns=["repo_name", "path"], num_rows=10)

In [ ]:
# JOIN the two tables on the repo_name

query = """
        SELECT sf.repo_name, COUNT(sc.commit) AS numb_commits
        FROM `bigquery-public-data.github_repos.sample_files` AS sf
        INNER JOIN `bigquery-public-data.github_repos.sample_commits` AS sc ON sf.repo_name = sc.repo_name
        WHERE sf.path LIKE '%.py'
        GROUP BY sf.repo_name
        ORDER BY numb_commits DESC
"""

repos = github.query_to_pandas_safe(query, max_gb_scanned=5.5)

In [ ]:
repos